# declarations

In [1]:
headerFile='safe_app.h'

In [2]:
headerFile='safe_authenticator.h'

In [3]:
import subprocess

In [4]:
def preprocess(source):
    return subprocess.check_output(['gcc', '-E', '-P', '-'],
                          input=source,
                          universal_newlines=True)

In [5]:
with open(headerFile) as f:
    fileContent=f.read()

In [6]:
includes = preprocess(fileContent)

print(preprocess(fileContent))

In [7]:
typedefs=''
functionDeclarations=''
for i in includes.splitlines():
    if i[:4] == 'void':
        functionDeclarations+='\n'+i
    else:
        typedefs+='\n'+i

print(typedefs)

### dataTypes

In [8]:
from cffi import FFI
ffi = FFI()

In [9]:
import re

In [10]:
oneliners=re.compile('typedef[^{}]+?;')

In [11]:
myStructTypes=re.compile('typedef struct .+?}.+?;', flags=re.DOTALL)

In [12]:
allOneLiners=oneliners.findall(typedefs)

In [13]:
arrayTypes={'typedef uint8_t const* AsymPublicKey;':'typedef uint8_t AsymPublicKey[32];',
 'typedef uint8_t const* AsymSecretKey;':'typedef uint8_t AsymSecretKey[32];',
 'typedef uint8_t const* AsymNonce;':'typedef uint8_t AsymNonce[24];',
 'typedef uint8_t const* SymSecretKey;':'typedef uint8_t SymSecretKey[32];',
 'typedef uint8_t const* SymNonce;':'typedef uint8_t SymNonce[24];',
 'typedef uint8_t const* SignPublicKey;':'typedef uint8_t SignPublicKey[32];',
 'typedef uint8_t const* SignSecretKey;':'typedef uint8_t SignSecretKey[64];',
 'typedef uint8_t const* XorNameArray;':'typedef uint8_t XorNameArray[32];'}

In [14]:
for index,oneLiner in enumerate(allOneLiners):
    if oneLiner in arrayTypes.keys():
        allOneLiners[index] = arrayTypes[oneLiner]

print('\n'.join(allOneLiners))

In [15]:
oneLinerDeclaration = '\n'.join(allOneLiners)

In [16]:
ffi.cdef(oneLinerDeclaration)

In [17]:
ffi.new('AsymPublicKey*')

<cdata 'unsigned char(*)[32]' owning 32 bytes>

In [18]:
allMyStructs=myStructTypes.findall(typedefs)
declarationStructs=[]
for oneStruct in allMyStructs:
    newStruct = re.sub('.*\{','typedef struct {',oneStruct).replace(' const*','*')
    declarationStructs.append(newStruct)

In [19]:
headerFile[:-2]+'_datatype_declarations'

'safe_authenticator_datatype_declarations'

In [20]:
with open(headerFile[:-2]+'_datatype_declarations','w') as f:
    f.write(oneLinerDeclaration)
    for i in range(100):
        for oneDeclaration in declarationStructs:
            try:
                ffi.cdef(oneDeclaration)
                f.write(oneDeclaration+'\n')
                declarationStructs.remove(oneDeclaration)
            except:
                pass

### lib functions

print(functionDeclarations.replace(' const*','*'))

In [21]:
correctedFunctions=functionDeclarations.replace(' const*','*')

In [22]:
ffi.cdef(correctedFunctions)

In [23]:
with open(headerFile[:-2]+'_function_declarations','w') as f:
    f.write(correctedFunctions)